In [2]:
#YOUTUBE SUMMARIZER PROJECT

'''PROBLEM STATEMENT : Building a youtube summarizer that will summarize the content of the youtube video for saving time by quicker understanding or summarization about the video in a short time'''

'PROBLEM STATEMENT : Building a youtube summarizer that will summarize the content of the youtube video for saving time by quicker understanding or summarization about the video in a short time'

In [3]:
pip install youtube_transcript_api

In [4]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
#importing required modules and libraries

import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
import re
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
link = input("Enter the YouTube link: ")
unique_id = link.split("=")[-1]
sub = YouTubeTranscriptApi.get_transcript(unique_id)  
subtitle = " ".join([x['text'] for x in sub])

Enter the YouTube link: https://www.youtube.com/watch?v=iCvmsMzlF7o


In [3]:
from nltk.tokenize import sent_tokenize

In [4]:
subtitle = subtitle.replace("\n"," ")
sentences = sent_tokenize(subtitle)

In [5]:
organized_sent = {k:v for v,k in enumerate(sentences)}

In [6]:
tf_idf = TfidfVectorizer(min_df=2, 
                                    strip_accents='unicode',
                                    max_features=None,
                                    lowercase = True,
                                    token_pattern=r'w{1,}',
                                    ngram_range=(1, 3), 
                                    use_idf=1,
                                    smooth_idf=1,
                                    sublinear_tf=1,
                                    stop_words = 'english')

In [7]:
import numpy as np
sentence_vectors = tf_idf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

C:\Users\Veena V\anaconda3\lib\site-packages\sklearn\utils\_param_validation.py:541: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
C:\Users\Veena V\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['w'] not in stop_words.
  warnings.warn(


In [8]:
N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

In [9]:
# mapping the scored sentences with their indexes as in the subtitle
mapped_sentences = [(sentence,organized_sent[sentence]) for sentence in top_n_sentences]
# Ordering the top-n sentences in their original order
mapped_sentences = sorted(mapped_sentences, key = lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]
# joining the ordered sentence
summary = " ".join(ordered_sentences)

In [10]:
summary

'When I was a young researcher, doctoral student, my first year, I had a research professor who said to us, "Here\'s the thing, if you cannot measure it, it does not exist." Because what we do is we take fat from our butts and put it in our cheeks. We pretend like what we\'re doing doesn\'t have a huge impact on other people.'

In [11]:
!pip install transformers

In [11]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [12]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [13]:
input_tensor = tokenizer.encode( subtitle, return_tensors="pt", max_length=512)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [14]:
outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
outputs_tensor

tensor([[    2,     0,   104,  2368,   229, 17648,    35,    38,   437,    10,
         29981,  9338,     4,    38,  5555,  1652,   131,    14,    18,    99,
            38,   109,     4,   178,  2085,  1652,    32,    95,   414,    19,
            10,  7047,     4,   178,    98,    38,    26,     6,    22,  1185,
           216,    99,   116,  2612,   218,    75,    47,    95,   486,   162,
            10,  9338,    12,  6462,   859, 12853,  1917,   178,    79,   439,
             6,    22, 24017,  2489,     4,   345,    18,   117,   215,   631,
            72,    36,   574, 35609,    43,   407,    38,   437,   164,     7,
          1067,     7,    47,   452,   480,    52,   214,  1686,    59,  5222,
         10518,   480,     8,    98,    38,   236,     7,  1137,   103,  1652,
            59,    10,  2125,     9,   127,   557,    14, 16894,  4939,   127,
         10518,     8,   269,   888,  1714,     5,   169,    14,    38,   697,
             8,   657,     8,   173,     8,  4095,  

In [15]:
print(tokenizer.decode(outputs_tensor[0]))

</s><s>Sally Kohn: I'm a qualitative researcher. I collect stories; that's what I do. And maybe stories are just data with a soul. And so I said, "You know what? Why don't you just call me a researcher-storyteller?" And she went, "Ha ha. There's no such thing." (Laughter) So I'm going to talk to you today -- we're talking about expanding perception -- and so I want to tell some stories about a piece of my research that fundamentally expanded my perception and really actually changed the way that I live and love and work and parent.</s>
